<a href="https://colab.research.google.com/github/gafbf/AI-Art-101/blob/main/ChatBot_Teste_00.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

# specify the path to your ebook directory
ebook_directory = '/content/drive/MyDrive/ebook_directory'

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
!pip install transformers
!pip install Flask
!pip install torch
!pip install PyDrive



Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [3]:
import os
import re

def preprocess_text(text):
    # Remove page numbers and other metadata
    text = re.sub(r'\d+', '', text)
    # Remove non-alphanumeric characters
    text = re.sub(r'\W+', ' ', text)
    # Remove excess whitespace
    text = re.sub(r'\s+', ' ', text)
    return text

def read_text_file(file_path):
    with open(file_path, 'r') as file:
        text = file.read()
    return preprocess_text(text)

def read_ebook_directory(directory_path):
    ebook_texts = []
    for root, dirs, files in os.walk(directory_path):
        for file in files:
            if file.endswith('.txt'):
                file_path = os.path.join(root, file)
                ebook_texts.append(read_text_file(file_path))
    return ebook_texts

In [4]:
from transformers import TextDataset, DataCollatorForLanguageModeling, AutoTokenizer

# Set the paths
ebook_directory = '/content/drive/MyDrive/ebook_directory'
output_directory = '/content/drive/MyDrive/output'

# Create the output directory if it doesn't exist
os.makedirs(output_directory, exist_ok=True)

# Load the tokenizer
tokenizer = AutoTokenizer.from_pretrained('gpt2')

# Read the ebook texts
ebook_texts = []
for ebook_file in os.listdir(ebook_directory):
    file_path = os.path.join(ebook_directory, ebook_file)
    if os.path.isfile(file_path):
        with open(file_path, 'r') as f:
            ebook_texts.append(f.read())

# Join the ebook texts into one long string
ebook_text = '\n'.join(ebook_texts)

# Save the ebook text to a file
with open(os.path.join(output_directory, 'ebook.txt'), 'w') as f:
    f.write(ebook_text)

# Convert the ebook text into a TextDataset
dataset = TextDataset(
    tokenizer=tokenizer,
    file_path=os.path.join(output_directory, 'ebook.txt'),
    block_size=128,
)

# Prepare the data for training
data_collator = DataCollatorForLanguageModeling(
    tokenizer=tokenizer, mlm=False,
)


/usr/local/lib/python3.9/dist-packages/transformers/data/datasets/language_modeling.py:53: FutureWarning: This dataset will be removed from the library soon, preprocessing should be handled with the 🤗 Datasets library. You can have a look at this example script for pointers: https://github.com/huggingface/transformers/blob/main/examples/pytorch/language-modeling/run_mlm.py
  warnings.warn(


In [5]:
from torch.utils.data import random_split

# Split the dataset into training and validation sets
train_size = int(0.8 * len(dataset))
val_size = len(dataset) - train_size
train_dataset, val_dataset = random_split(dataset, [train_size, val_size])


In [8]:
import torch
from transformers import AutoModelWithLMHead, AutoTokenizer
from google.colab import auth
from oauth2client.client import GoogleCredentials
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive

# Authenticate and create PyDrive client
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

# ID of the file to download
file_id = 'your_file_id_here'

# Download the file
downloaded = drive.CreateFile({'id': file_id})
downloaded.GetContentFile('filename.ext')


# Load the trained model and tokenizer
output_directory = '/content/drive/MyDrive/output'
model_checkpoint = 'gpt2'
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)
model = AutoModelWithLMHead.from_pretrained(output_directory)

# Set the device to use for inference
device = 'cuda' if torch.cuda.is_available() else 'cpu'
model.to(device)

# Define a function to generate text from the model
def generate_text(prompt, max_length=200):
    input_ids = tokenizer.encode(prompt, return_tensors='pt').to(device)
    output = model.generate(
        input_ids=input_ids,
        max_length=max_length,
        do_sample=True,
        top_k=50,
        top_p=0.95,
        temperature=1.0,
    )
    return tokenizer.decode(output[0], skip_special_tokens=True)

# Example usage
prompt = "What is the meaning of life?"
generated_text = generate_text(prompt)
print(generated_text)


FileNotDownloadableError: ignored